# How to use

## In Google Coalbo
1. Download this notebook
2. Uploda to Google Coalbo
3. Excute


## In Local

NOTICE: Required CUDA 12.2, Run on Windows 11 64bit
1. Clone Repository
```bash
git clone git@github.com:rtr-x8/GNNMF-CL.git
```
2. Create conda environment by Anaconda Prompt
```bash
conda create -n GNNMF-CL python=3.11.11 -y
conda activate GNNMF-CL
conda install -c anaconda ipykernel -y
python -m ipykernel install --user --name GNNMF-CL --display-name "GNNMF-CL Env"

```
3. Select Conda Environment in your Edirot(ex. VSCODE)


In [1]:
!python --version
# expect Python 3.11.11

Python 3.11.11


In [2]:
# Run Once
"""
!pip uninstall torch -y
!pip install -q --no-cache-dir torch==2.4.1 --index-url https://download.pytorch.org/whl/cu121
!pip uninstall torch-scatter torch-sparse pyg-lib torch-geometric torchvision -y
!pip install -q --no-cache-dir torch-geometric
!pip install -q --no-cache-dir pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.4.0+cu121.html
!pip install -q --no-cache-dir torchvision==0.19.1 --index-url https://download.pytorch.org/whl/cu121
!pip -q --no-cache-dir install torchmetrics

!pip install jupyter_contrib_nbextensions
!pip install --upgrade ipywidgets
!jupyter contrib nbextension install --user
!jupyter nbextension enable --py widgetsnbextension

!pip install scikit-learn tqdm pandas numpy setuptools datetime pytz sentence-transformers matplotlib seaborn
!pip install -q python-dotenv wandb
!pip install torchinfo
"""

import torch
import torch_geometric
import torch_scatter
import torch_sparse
import pyg_lib
import torchvision
import torchmetrics

In [3]:
import sys
import os


if "vingat" in sys.modules:
    del sys.modules["vingat"]

sys.path.append('..')

from vingat.logger import get_run_name, log_metrics
from vingat.assertion import assert_package_versions
from vingat.loader import (
    core_file_loader, load_recipe_nutrients, load_ingredients,
    load_recipe_ingredients, load_recipe_cooking_directions,
    load_ingredients_with_embeddings, load_recipe_image_embeddings, load_recipe_image_embeddings_ft,
    load_recipe_image_vlm_caption,
    load_recipe_cooking_directions_embeddings,
    load_recipe_image_vlm_caption_embeddings,
    load_user_embeddings, load_alternative_ingredients,
    train_dataclustering, calculate_cluster, load_clouster_centers)
from vingat.loss import BPRLoss
from vingat.model import RecommendationModel
from vingat.functions import evaluate_model, save_model, train_func, get_item_popularity
from vingat import __version__ as vingat_version
from vingat.dataloader import create_base_hetero, mask_hetero, create_dataloader
from vingat.visualizer import visualize_node_pca
from vingat.preprocess import filter_recipe_ingredient, NutrientStandardPreprocess

os.environ['CUDA_LAUNCH_BLOCKING']="1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"
assert_package_versions() # assert versions


In [4]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [5]:
from dotenv import load_dotenv
load_dotenv()

import os
import wandb

wandab_api = os.getenv('WANDB_API')

In [6]:
from torch_geometric.data import HeteroData
from sklearn.preprocessing import LabelEncoder
import torch
from enum import Enum
import numpy as np
from vingat.loader import use_nutritions
import pandas as pd
from torch_geometric.loader import LinkNeighborLoader

import pandas as pd
import json


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [7]:
import random
PROJECT_NAME = "vingat-v3_local"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("device: ", device)

CONFIG = {
    "architecture": "LightGCN, HGT, GAT",
    "_mode" : "in_local",
    "batch_size": 256,
    "bprloss_reg_lambda": 0.001,
    "cl_loss_rate": 0.2,
    "cluster_margin": 0.5,
    "cluster_loss_weight": 1.2,
    "criterion": "",
    "default_decay": 0.00002,
    "device": device,
    "dropout_rate": 0.3,
    "epochs": 20,
    "filter_ingredient_sim_score": 0.7,
    "fusion_gnn_after_dropout_rate": 0.2,
    "fusion_gnn_dropout_rate": 0.2,
    "fusion_gnn_resisual_alpha": 0.5,
    "fusion_layers": 1,
    "hidden_dimention": 128,
    "image_encoder_low_rank_dim": 64,
    "input_cooking_direction_dim": 384,
    "input_image_dim": 1024,
    "input_ingredient_dim": 384,
    "input_vlm_caption_dim": 384,
    "intention_cl_after_dropout_rate": 0.2,
    "intention_layers": 1,
    "item_encoder_dropout_rate": 0.2,
    "item_encoder_low_rank_dim": 64,
    "learning_rate": 0.00002,
    "link_predictor_dropout_rate": 0.2,
    "link_predictor_leaky_relu_slope": 0.3,
    "max_grad_norm": 30,
    "multi_head": 1,
    "node_embeding_dimmention": 32,
    "nutrient_dim": 20,
    "patience": 20,  #  Early stop at least, * validation_interval
    "pyg_lib v": pyg_lib.__version__,
    "rating_threshold": 3.5,
    "scheduler_gamma": 0.975,
    "scheduler_size": 10,
    "seed": 2020,
    "sencing_layers": 1,
    "sensing_gnn_resisual_alpha": 0.5,
    "taste_gnn_after_dropout_rate": 0.2,
    "taste_gnn_dropout_rate": 0.2,
    "temperature": 0.1,
    "torch v": torch.__version__,
    "torch_geometric v": torch_geometric.__version__,
    "torch_scatter v": torch_scatter.__version__,
    "torch_sparse v": torch_sparse.__version__,
    "user_encoder_dropout_rate": 0.2,
    "user_encoder_low_rank_dim": 64,
    "user_encoder_weight_decay": 0.000001,
    "validation_interval": 5,
    "vingat_v": vingat_version,

}

device:  cuda:0


In [8]:
random.seed(CONFIG["seed"])
np.random.seed(CONFIG["seed"])
torch.manual_seed(CONFIG["seed"])
torch.cuda.manual_seed_all(CONFIG["seed"])

run_name = get_run_name()
run_name = f"{run_name}_{vingat_version}"

In [9]:
wandb.login(key=wandab_api)
wandb.init(
  project=PROJECT_NAME,
  name=run_name,
  config=CONFIG,
  tags=["in_local", "use_mini_data"]
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\matsuoka\_netrc
wandb: Currently logged in as: ryu-2-24 (ryu-2-24-shiga-u-ac-jp) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [10]:
PATH = os.path.join(os.getcwd(), '..', 'data', 'mini')

# File loader
core_recipes, core_train_rating, core_test_rating, core_val_rating = core_file_loader(PATH, CONFIG["rating_threshold"])
core_recipe_indices = core_recipes.index.values
recipe_nutrients = load_recipe_nutrients(PATH, core_recipes.copy())
ingredients = load_ingredients(PATH, core_recipes.copy())
alternative_ingredients = load_alternative_ingredients(PATH, core_recipes.copy(), CONFIG["device"])
recipe_ingredients = load_recipe_ingredients(PATH, core_recipes.copy())
recipe_cooking_directions = load_recipe_cooking_directions(PATH, core_recipes.copy())
ingredients_with_embeddings = load_ingredients_with_embeddings(PATH, ingredients.copy())
recipe_image_embeddings = load_recipe_image_embeddings_ft(PATH, core_recipes.copy(), CONFIG["device"])
recipe_image_vlm_caption = load_recipe_image_vlm_caption(PATH)
recipe_cooking_directions_embeddings = load_recipe_cooking_directions_embeddings(PATH, recipe_cooking_directions.copy())
recipe_ingredients = filter_recipe_ingredient(recipe_ingredients, alternative_ingredients, CONFIG["filter_ingredient_sim_score"])
ingredients = ingredients[ingredients.index.isin(recipe_ingredients["ingredient_id"])]
recipe_image_vlm_caption_embeddings = load_recipe_image_vlm_caption_embeddings(PATH, recipe_image_vlm_caption.copy())

train_nutrients, kmeans_model, scaler = train_dataclustering(
    train_data=recipe_nutrients.loc[recipe_nutrients.index.isin(core_train_rating.recipe_id)][use_nutritions],
    n_cluster=6,
    path=PATH
)
test_recipe_ids = set(core_train_rating.recipe_id) - set()
test_nutrients = calculate_cluster(
    data=recipe_nutrients.loc[recipe_nutrients.index.isin(core_test_rating.recipe_id)][use_nutritions],
    path=PATH,
    scaler=scaler,
    kmeans_model=kmeans_model
)
valid_nutrients = calculate_cluster(
    data=recipe_nutrients.loc[recipe_nutrients.index.isin(core_val_rating.recipe_id)][use_nutritions],
    path=PATH,
    scaler=scaler,
    kmeans_model=kmeans_model
)
recipe_nutrients = pd.concat([
    train_nutrients, test_nutrients, valid_nutrients
])
recipe_nutrients = recipe_nutrients.loc[~recipe_nutrients.index.duplicated(keep='first')]
recipe_cluster_centers = load_clouster_centers(kmeanth_model=kmeans_model, device=device)

train_nutrients.describe()

一旦使わないけど： 3.5
recipe_nutrients is loaded
ingredients is loaded
recipe_ingredients is loaded
recipe_cooking_directions is loaded
ingredients_with_embeddings is loaded
recipe_image_embeddings_ft is loaded


,niacin,fiber,sugars,sodium,carbohydrates,vitaminB6,calories,thiamin,fat,folate,...,calcium,magnesium,iron,cholesterol,protein,vitaminA,potassium,saturatedFat,vitaminC,cluster
count,6950.000000,6950.000000,6950.000000,6950.000000,6950.000000,6950.000000,6950.000000,6950.000000,6950.000000,6950.000000,...,6950.000000,6950.000000,6950.000000,6950.000000,6950.000000,6950.000000,6950.000000,6950.000000,6950.000000,6950.000000
mean,6.056265,2.424388,11.385730,580.987665,29.728396,0.275931,326.584480,0.200921,17.051330,48.529974,...,102.940808,35.114430,2.136509,60.042701,14.174080,1289.980934,356.040175,6.571585,12.649274,1.775827
std,6.468247,2.654089,13.746483,1465.137651,21.862955,0.296557,196.103015,0.288020,13.471687,51.677263,...,124.708199,29.992323,1.993833,63.393908,13.830219,3283.366113,325.277145,6.180946,23.611676,1.192432
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.471414,0.678698,2.265741,170.983575,12.787170,0.049621,181.955825,0.064685,7.048789,14.615028,...,27.884135,12.998015,0.907851,12.201250,3.382456,142.403300,107.221800,1.966310,0.476187,1.000000
50%,3.383996,1.631601,5.852115,380.589800,26.327295,0.178652,297.704250,0.135401,13.992975,33.393005,...,59.692890,28.392865,1.637120,43.648125,8.424019,406.498100,274.347200,4.861394,3.752231,1.000000
75%,8.999851,3.247825,16.288235,793.228300,42.091723,0.411310,440.136750,0.247910,23.913720,62.913097,...,123.267650,49.411653,2.931236,86.325675,23.141918,1000.386600,508.803725,9.475899,15.077220,2.000000
max,46.788720,30.841200,142.608000,91282.320000,236.720500,3.685958,1828.192000,13.135970,128.705300,597.352000,...,1178.472000,435.951300,54.684120,601.299600,122.998300,72542.200000,3301.576000,50.596670,661.818500,5.000000


In [11]:
set([1, 2, 3, 4, 5]) - set([4, 5, 6, 7])

{1, 2, 3}

In [12]:
nutri_scalar = NutrientStandardPreprocess(use_nutrients=use_nutritions)
recipe_nutrients = nutri_scalar.do(
  train_recipe_ids=core_train_rating.recipe_id.unique(),
  test_recipe_ids=core_test_rating.recipe_id.unique(),
  val_recipe_ids=core_val_rating.recipe_id.unique(),
  recipe_nutrients=recipe_nutrients
)
recipe_cluster_centers = nutri_scalar.transform_from_tensor(recipe_cluster_centers)
recipe_nutrients.loc[recipe_nutrients.index.isin(core_train_rating.recipe_id), "cluster"].value_counts()

c:\Users\matsuoka\.conda\envs\GNNMF-CL\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


cluster
1    2787
2    1726
4     947
3     770
0     719
5       1
Name: count, dtype: int64

In [13]:
#create mini data
"""
mini_path = os.path.join(os.getcwd(), '..', 'data', 'mini')
mini_core_train_rating = core_train_rating.sample(frac=0.02, random_state=1)
mini_core_test_rating = core_test_rating.sample(frac=0.02, random_state=1)
mini_core_val_rating = core_val_rating.sample(frac=0.02, random_state=1)
user_ids = pd.concat([mini_core_train_rating, mini_core_test_rating, mini_core_val_rating]).user_id.unique()
recipe_ids = pd.concat([mini_core_train_rating, mini_core_test_rating, mini_core_val_rating]).recipe_id.unique()
mini_core_recipes = core_recipes.loc[core_recipes.index.isin(recipe_ids)]
mini_recipe_ingredients = recipe_ingredients.loc[recipe_ingredients["recipe_id"].isin(recipe_ids)]
mini_ingredients = ingredients.loc[ingredients.index.isin(mini_recipe_ingredients["ingredient_id"])]
mini_recipe_nutrients = recipe_nutrients.loc[recipe_nutrients.index.isin(recipe_ids)]
mini_recipe_image_embeddings = recipe_image_embeddings.loc[recipe_image_embeddings.index.isin(recipe_ids)]
mini_recipe_image_vlm_caption_embeddings = recipe_image_vlm_caption_embeddings.loc[recipe_image_vlm_caption_embeddings.index.isin(recipe_ids)]
mini_recipe_cooking_directions_embeddings = recipe_cooking_directions_embeddings.loc[recipe_cooking_directions_embeddings.index.isin(recipe_ids)]
mini_ingredients_with_embeddings = ingredients_with_embeddings.loc[ingredients_with_embeddings.index.isin(mini_ingredients.index)]
mini_recipe_cooking_directions = recipe_cooking_directions.loc[recipe_cooking_directions.index.isin(recipe_ids)]
mini_recipe_image_vlm_caption = recipe_image_vlm_caption.loc[recipe_image_vlm_caption.index.isin(recipe_ids)]
mini_alternative_ingredients = load_alternative_ingredients(mini_path, mini_core_recipes.copy(), CONFIG["device"])
mini_recipe_ingredients = filter_recipe_ingredient(mini_recipe_ingredients, mini_alternative_ingredients, CONFIG["filter_ingredient_sim_score"])
mini_ingredients = mini_ingredients[mini_ingredients.index.isin(mini_recipe_ingredients["ingredient_id"])]

diff_ing_id = list(set(mini_recipe_ingredients.ingredient_id) - set(mini_ingredients.index))
mini_recipe_ingredients = mini_recipe_ingredients.loc[~mini_recipe_ingredients["ingredient_id"].isin(diff_ing_id)]

mini_core_train_rating.to_csv(os.path.join(mini_path, "core-data-train_rating.csv"))
mini_core_test_rating.to_csv(os.path.join(mini_path, "core-data-test_rating.csv"))
mini_core_val_rating.to_csv(os.path.join(mini_path, "core-data-valid_rating.csv"))
mini_recipe_ingredients.to_csv(os.path.join(mini_path, "recipe_ingredients.csv"))
mini_ingredients.to_csv(os.path.join(mini_path, "ingredients.csv"))
mini_recipe_image_embeddings.to_csv(os.path.join(mini_path, "recipe_image_embeddings_ft.csv"))
mini_recipe_image_vlm_caption_embeddings.to_csv(os.path.join(mini_path, "recipe_image_vlm_caption_embeddings.csv"))
mini_recipe_cooking_directions_embeddings.to_csv(os.path.join(mini_path, "recipe_cooking_directions_embeddings.csv"))
mini_ingredients_with_embeddings.to_csv(os.path.join(mini_path, "ingredients_embeddings.csv"))
mini_core_recipes.to_csv(os.path.join(mini_path, "core-data_recipe.csv"))
mini_recipe_nutrients.to_csv(os.path.join(mini_path, "recipe_nutrients.csv"))
mini_alternative_ingredients.to_csv(os.path.join(mini_path, "alternative_ingredients.csv"))
mini_recipe_cooking_directions.to_csv(os.path.join(mini_path, "recipe_cooking_directions.csv"))
mini_recipe_image_vlm_caption.to_csv(os.path.join(mini_path, "recipe_image_vlm_caption.csv"))

"""

recipe_nutrients.loc[recipe_nutrients.index.isin(core_train_rating["recipe_id"])].describe()

,niacin,fiber,sugars,sodium,carbohydrates,vitaminB6,calories,thiamin,fat,folate,...,calcium,magnesium,iron,cholesterol,protein,vitaminA,potassium,saturatedFat,vitaminC,cluster
count,6950.000000,6950.000000,6950.000000,6950.000000,6950.000000,6950.000000,6950.000000,6950.000000,6950.000000,6950.000000,...,6950.000000,6950.000000,6950.000000,6950.000000,6950.000000,6950.000000,6950.000000,6950.000000,6950.000000,6950.000000
mean,-0.025976,0.013982,0.007541,0.001310,0.012673,-0.013478,-0.011317,-0.007633,-0.017516,0.004639,...,0.002759,0.005472,-0.006643,0.002265,-0.021279,0.024753,0.004460,-0.006510,0.011899,1.775827
std,0.982712,1.031014,1.010347,0.908836,1.013802,1.008062,1.001939,0.979055,0.997697,1.001132,...,1.011025,1.014352,1.000165,1.019826,0.998800,1.105704,1.027924,1.012200,1.036524,1.192432
min,-0.936374,-0.913520,-0.828325,-0.396570,-1.359859,-0.930517,-1.665492,-0.697644,-1.265807,-0.939165,...,-0.825513,-1.170865,-1.071636,-0.947205,-1.024937,-0.392912,-1.094654,-1.063277,-0.535760,0.000000
25%,-0.713698,-0.650320,-0.662233,-0.279714,-0.763293,-0.766454,-0.741613,-0.476878,-0.756425,-0.650740,...,-0.601069,-0.741064,-0.619398,-0.758764,-0.780349,-0.349586,-0.769784,-0.752970,-0.516394,1.000000
50%,-0.429341,-0.300922,-0.400907,-0.136887,-0.141082,-0.356771,-0.175164,-0.227107,-0.241294,-0.284061,...,-0.346481,-0.237817,-0.254347,-0.272620,-0.440149,-0.266919,-0.269504,-0.296145,-0.377708,1.000000
75%,0.386954,0.315672,0.375899,0.145139,0.563400,0.428781,0.552770,0.152542,0.481001,0.298963,...,0.183573,0.478140,0.365791,0.417492,0.598009,-0.077605,0.462138,0.456211,0.111517,2.000000
max,6.146846,10.707574,9.546566,61.910810,9.468391,11.499567,7.657789,44.913399,10.657285,10.620948,...,17.716952,10.386037,26.356965,7.606152,7.869149,21.702528,9.056117,10.187822,27.495546,5.000000


In [14]:
# pd.DataFrame(nutri_scalar.transform(recipe_nutrients.loc[recipe_nutrients.index.isin(core_train_rating["recipe_id"])])).describe()

In [15]:
# pd.DataFrame(recipe_cluster_centers.cpu().numpy()).describe()

In [24]:
#stop()

data, user_lencoder, item_lencoder, ing_lencoder = create_base_hetero(
    core_train_rating=core_train_rating,
    core_test_rating=core_test_rating,
    core_val_rating=core_val_rating,
    ingredients=ingredients,
    recipe_nutrients=recipe_nutrients,
    recipe_image_embeddings=recipe_image_embeddings,
    recipe_image_vlm_caption_embeddings=recipe_image_vlm_caption_embeddings,
    recipe_cooking_directions_embeddings=recipe_cooking_directions_embeddings,
    ingredients_with_embeddings=ingredients_with_embeddings,
    directory_path=PATH,
    device=CONFIG["device"],
    hidden_dim=CONFIG["hidden_dimention"],
    input_image_dim=CONFIG["input_image_dim"],
    input_vlm_caption_dim=CONFIG["input_vlm_caption_dim"],
    input_ingredient_dim=CONFIG["input_ingredient_dim"],
    input_cooking_direction_dim=CONFIG["input_cooking_direction_dim"],
)

train_data, ss = mask_hetero(data, core_train_rating, recipe_ingredients, user_lencoder, item_lencoder, ing_lencoder, is_train=True)
test_data, _ = mask_hetero(data, core_test_rating, recipe_ingredients, user_lencoder, item_lencoder, ing_lencoder, is_train=False, scalar_preprocess=ss)
val_data, _ = mask_hetero(data, core_val_rating, recipe_ingredients, user_lencoder, item_lencoder, ing_lencoder, is_train=False, scalar_preprocess=ss)

mini_train = test_data.clone()
pops = get_item_popularity(device, item_lencoder, PATH, CONFIG["rating_threshold"])

train_loader = create_dataloader(train_data, CONFIG["batch_size"], num_workers=2, neg_sampling_ratio=1.0, popularity=pops, is_abration_cl=False)
test_loader = create_dataloader(test_data, CONFIG["batch_size"], shuffle=False, neg_sampling_ratio=0.0, is_abration_cl=False)
val_loader = create_dataloader(val_data, CONFIG["batch_size"], shuffle=False, neg_sampling_ratio=0.0, is_abration_cl=False)


一旦使わないけど： 3.5


In [25]:
print("\n\n\nデータ構造")
train_data["intention"]["cluster"].unique()




データ構造


tensor([0, 1, 2, 3, 4, 5], device='cuda:0')

In [20]:
import torch.nn as nn

model = RecommendationModel(
    dropout_rate=CONFIG["dropout_rate"],
    device=CONFIG["device"],
    hidden_dim=CONFIG["hidden_dimention"],
    node_embeding_dimmention=CONFIG["node_embeding_dimmention"],
    num_user=len(user_lencoder.classes_),
    num_item=len(item_lencoder.classes_),
    nutrient_dim=CONFIG["nutrient_dim"],
    num_heads=CONFIG["multi_head"],
    sencing_layers=CONFIG["sencing_layers"],
    fusion_layers=CONFIG["fusion_layers"],
    intention_layers=CONFIG["intention_layers"],
    temperature=CONFIG["temperature"],
    cl_loss_rate=CONFIG["cl_loss_rate"],
    input_image_dim=CONFIG["input_image_dim"],
    input_vlm_caption_dim=CONFIG["input_vlm_caption_dim"],
    input_ingredient_dim=CONFIG["input_ingredient_dim"],
    input_cooking_direction_dim=CONFIG["input_cooking_direction_dim"],
    user_encoder_low_rank_dim=CONFIG["user_encoder_low_rank_dim"],
    item_encoder_low_rank_dim=CONFIG["item_encoder_low_rank_dim"],
    user_encoder_dropout_rate=CONFIG["user_encoder_dropout_rate"],
    item_encoder_dropout_rate=CONFIG["item_encoder_dropout_rate"],
    intention_cl_after_dropout_rate=CONFIG["intention_cl_after_dropout_rate"],
    taste_gnn_dropout_rate=CONFIG["taste_gnn_dropout_rate"],
    taste_gnn_after_dropout_rate=CONFIG["taste_gnn_after_dropout_rate"],
    fusion_gnn_dropout_rate=CONFIG["fusion_gnn_dropout_rate"],
    fusion_gnn_after_dropout_rate=CONFIG["fusion_gnn_after_dropout_rate"],
    link_predictor_dropout_rate=CONFIG["link_predictor_dropout_rate"],
    link_predictor_leaky_relu_slope=CONFIG["link_predictor_leaky_relu_slope"],
    sensing_gnn_resisual_alpha=CONFIG["sensing_gnn_resisual_alpha"],
    fusion_gnn_resisual_alpha=CONFIG["fusion_gnn_resisual_alpha"],
    is_abration_wo_cl=False,
    is_abration_wo_taste=False,
    image_encoder_low_rank_dim=CONFIG["image_encoder_low_rank_dim"],
    cluster_centers=recipe_cluster_centers,
    cluster_margin=CONFIG["cluster_margin"],
    cluster_loss_weight=CONFIG["cluster_loss_weight"],
)
model.to(device)

user_encoder_params = list(model.user_encoder.parameters())

other_params = [
    p for p in model.parameters()
    if not any(torch.equal(p.data, up.data) for up in user_encoder_params)
]

optimizer = torch.optim.Adam(
    [
        {
            "params": user_encoder_params,
            "weight_decay": CONFIG["user_encoder_weight_decay"],
        },
        {
            "params": other_params,
            "weight_decay": CONFIG["default_decay"],
        }
    ],
    lr=CONFIG["learning_rate"],
    # weight_decay=CONFIG["default_decay"],
)

# criterion = BPRLoss(reg_lambda=CONFIG["bprloss_reg_lambda"])
# criterion = nn.BCELoss()

scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=CONFIG["scheduler_size"],
    gamma=CONFIG["scheduler_gamma"]
)

In [21]:
from torchinfo import summary

model_summary = summary(model, verbose=0)

summary_text = str(model_summary)
summary_text = summary_text.replace("\n", "<br>")

wandb.log({"model_summary": wandb.Html(summary_text)})
print("\n\n\nモデルの構造")
print(model_summary)

wandb.watch(model, log="gradients", log_freq=100)




モデルの構造
Layer (type:depth-idx)                        Param #
RecommendationModel                           --
├─Sequential: 1-1                             --
│    └─Embedding: 2-1                         878,720
│    └─Linear: 2-2                            8,320
│    └─Dropout: 2-3                           --
├─Sequential: 1-2                             --
│    └─Embedding: 2-4                         655,424
│    └─Linear: 2-5                            8,320
│    └─Dropout: 2-6                           --
├─LowRankLinear: 1-3                          --
│    └─Linear: 2-7                            65,536
│    └─Linear: 2-8                            8,320
├─Linear: 1-4                                 49,280
├─Linear: 1-5                                 49,280
├─NutrientCaptionContrastiveLearning: 1-6     --
│    └─Sequential: 2-9                        --
│    │    └─Linear: 3-1                       1,344
│    │    └─ReLU: 3-2                         --
│    │    └─Linear: 

In [22]:
def wandb_logger(**kwargs):
    try:
        wandb.log(**kwargs)
    except:
        print(kwargs)
        pass

def wandb_tagger(*args):
    try:
        wandb.run.tags = list(wandb.run.tags) + list(args)
    except:
        print(args)
        pass

def wandb_scatter(df, step, title):
    df["step"] = step
    table = wandb.Table(data=df, columns=["PC1", "PC2", "node_type"])
    color_cahrt = wandb.plot.scatter(
        table,
        "PC1",
        "PC2",
        title=title,
    )
    wandb.run.log({f"scatter_step_{step}": color_cahrt},
                step=step)


pca_cols = ["intention", "taste", "image"]

criterion = BPRLoss(reg_lambda=CONFIG["bprloss_reg_lambda"])


train_func(
    train_loader,
    val_data,
    model,
    optimizer,
    scheduler,
    criterion,
    epochs=CONFIG["epochs"],
    device=device,
    wbLogger=wandb_logger,
    wbTagger=wandb_tagger,
    wbScatter=wandb_scatter,
    directory_path=PATH,
    project_name=PROJECT_NAME,
    experiment_name=run_name,
    popularities=pops,
    patience=CONFIG["patience"],
    validation_interval=CONFIG["validation_interval"],
    max_grad_norm=CONFIG["max_grad_norm"],
    pca_cols=pca_cols,
    recommendation_loss_weight=1
)




 Epoch 1/20 2025-03-21 15:18:08


[Train]:   0%|          | 0/53 [00:00<?, ?it/s]

1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size

[Train]:   0%|          | 0/53 [00:00<?, ?it/s]

1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size

...

1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size

[Train]:   0%|          | 0/53 [00:00<?, ?it/s]

1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size([1, 128]) tensor([0], device='cuda:0')
1 torch.Size

[Train]:   0%|          | 0/53 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
score_statics, t_mhandler = evaluate_model(
    model=model,
    data=test_data,
    device=CONFIG["device"],
    freq_tensor=pops,
    desc="[Test]"
)


print("Score Statics: ")
print(score_statics.log(prefix="test-score-statics", num_round=4))
wandb_logger(data=score_statics.log(prefix="test-score-statics"))

print("handler Result: ")
print(t_mhandler.log(prefix="test-handler", num_round=4))
wandb_logger(data=t_mhandler.log(prefix="test-handler", num_round=8))

In [ ]:
wandb.config["criterion"] = str(criterion.__class__.__name__)
wandb.run.tags = list(wandb.run.tags) + ["image_fine_tuned"]

#wandb.run.tags = list(wandb.run.tags) + ["crashed"]
#wandb.run.tags = list(wandb.run.tags) + ["all_same_result"]
#wandb.run.tags = list(wandb.run.tags) + ["manual_stopped"]
#wandb.run.tags = list(wandb.run.tags) + ["over_learning"]
#wandb.run.tags = list(wandb.run.tags) + ["not_learning"]
#wandb.run.tags = list(wandb.run.tags) + ["not_satisfied"]
#wandb.run.tags = list(wandb.run.tags) + ["is_trial_model"]
wandb.finish()